# 311 Data Exploration and Analysis

In [1]:
import pandas as pd
from sodapy import Socrata
import numpy as np
import datetime
from datetime import date
pd.set_option('display.max_rows', 1000)
pd.options.display.max_columns = None

In [2]:
socrata_domain = "data.cityofchicago.org"
socrata_dataset_identifier = "v6vf-nfxy"
app_token = "6tRoBirkYQMdr8MMFR8FzgBXq"
api_username = "v4vigtory@gmail.com"
api_password = "311_TeamVeam"

In [3]:
client = Socrata(socrata_domain, app_token, username=api_username, password=api_password)

### Adding Data from 2019~2021

In [4]:
sr_2021 = client.get(socrata_dataset_identifier,
                     select = "sr_number, sr_type, sr_short_code, owner_department, status, created_date, closed_date, date_extract_y(created_date) as year, street_address, city, state, zip_code, community_area, location",
                     where = "date_extract_y(created_date) = 2021 AND community_area IS NOT NULL AND sr_type NOT IN ('311 INFORMATION ONLY CALL', 'Aircraft Noise Complaint')", 
                     limit = 1000000)
sr_2021_df = pd.DataFrame(sr_2021)

In [5]:
sr_2020 = client.get(socrata_dataset_identifier,
                     select = "sr_number, sr_type, sr_short_code, owner_department, status, created_date, closed_date, date_extract_y(created_date) as year, street_address, city, state, zip_code, community_area, location",
                     where = "date_extract_y(created_date) = 2020 AND community_area IS NOT NULL AND sr_type NOT IN ('311 INFORMATION ONLY CALL', 'Aircraft Noise Complaint')", 
                     limit = 1000000)
sr_2020_df = pd.DataFrame(sr_2020)

In [6]:
sr_2019 = client.get(socrata_dataset_identifier,
                     select = "sr_number, sr_type, sr_short_code, owner_department, status, created_date, closed_date, date_extract_y(created_date) as year, street_address, city, state, zip_code, community_area, location",
                     where = "date_extract_y(created_date) = 2019 AND community_area IS NOT NULL AND sr_type NOT IN ('311 INFORMATION ONLY CALL', 'Aircraft Noise Complaint')", 
                     limit = 1000000)
sr_2019_df = pd.DataFrame(sr_2019)

In [7]:
sr_2019_21_df = pd.concat([sr_2021_df,sr_2020_df,sr_2019_df])

In [9]:
sr_2019_21_df.shape

(2415606, 14)

### Findings from Exploratory Analysis (List of Assumptions / Things to Note)
- Got rid of two specific service request types (sr_type)
    - 311 Informational Calls: this is a 311 Call Center. These requests are informational not relevant to our analysis.
    - aircraft noise complaint: this maps to the Aviation Administration building, and exclusively deals with aircraft noise complaints. This is either geo-tagged to a single building and doesn't provide required variation across neighborhoods or are all completed within 0.01667 minutes. 


- The data will look like service requests for 2019 have a higher closed ratio than requests in 2021. 
    - max resolution time is around ~3 years 
    - many of these long-tailed requests are restaurant/health-related complaints
    - 17 rows of 3+ years 
    - 2382 rows of 2+ years data 
    - 35121 rows of 1+ years data (~1.4% of total data)
    
    
- Removed rows where request_closed_date - request_created_date (diff_min) was <= 0. Below are the service request types associated with diff_min < 0.
    - Vehicle Parked in Bike Lane Complaint    3286
    - E-Scooter                                 455
    - Extreme Weather Notification               25
    - Graffiti Removal Request                    1

### Creating Flags

In [29]:
df = sr_2019_21_df.copy()

In [31]:
# resol_time
df['diff_mins'] = (pd.to_datetime(df['closed_date']) - pd.to_datetime(df['created_date']))/np.timedelta64(1,'m')

# time_open_not_resolved
df['time_open_unresolved'] = np.where((df['diff_mins'].isna()),1, 0)

In [33]:
df = df[(df.diff_mins > 0) | (df.time_open_unresolved == 1)]

In [34]:
# time_open_less_than_1_min
df['time_open_less_than_1_min'] = np.where(df['diff_mins'] < 1, 1, 0)

# time_open_1_min_1_hr
df['time_open_1_min_1_hr'] = np.where((df['diff_mins'] >= 1) & (df['diff_mins'] < 60), 1, 0)

# time_open_1_hr_12_hr
df['time_open_1_hr_12_hr'] = np.where((df['diff_mins'] >= 60) & (df['diff_mins'] < 60*12), 1, 0)

# time_open_12_24_hr
df['time_open_12_24_hr'] = np.where((df['diff_mins'] >= 60*12) & (df['diff_mins'] < 60*24), 1, 0)

# time_open_1_3_day
df['time_open_1_3_day'] = np.where((df['diff_mins'] >= 60*24*1) & (df['diff_mins'] < 60*24*3), 1, 0)

# time_open_3_7_day
df['time_open_3_7_day'] = np.where((df['diff_mins'] >= 60*24*3) & (df['diff_mins'] < 60*24*7), 1, 0)

# time_open_7_14_day
df['time_open_7_14_day'] = np.where((df['diff_mins'] >= 60*24*7) & (df['diff_mins'] < 60*24*14), 1, 0)

# time_open_14_30_day
df['time_open_14_30_day'] = np.where((df['diff_mins'] >= 60*24*14) & (df['diff_mins'] < 60*24*30), 1, 0)

# time_open_1_3_month
df['time_open_1_3_month'] = np.where((df['diff_mins'] >= 60*24*30*1) & (df['diff_mins'] < 60*24*30*3), 1, 0)

# time_open_3_12_month
df['time_open_3_12_month'] = np.where((df['diff_mins'] >= 60*24*30*3) & (df['diff_mins'] < 60*24*30*12), 1, 0)

# time_open_1_year_plus
df['time_open_1_year_plus'] = np.where((df['diff_mins'] >= 60*24*30*12),1, 0)

# code snippet
#df.filter(regex="time_open_", axis = 1).sum()

## Data Analysis Portion Left to Do

In [86]:
# THINGS TO DO FOR ANALYSIS AFTER CREATING DASH 

# spread of service request types 
# unresolved across neighborhoods
# .corr grid for analysis 

nan

## Analysis of Unresolved Requests

In [34]:
# create column 'time_open'
df['time_open'] = ((pd.to_datetime("today")) - pd.to_datetime(df['created_date']))/np.timedelta64(1,'m')

In [ ]:
# count of unresolved across community areas
community_area